In [1]:
import pandas as pd
import anthropic

In [2]:
MODELS = dict(haiku="claude-3-haiku-20240307", sonnet="claude-3-5-sonnet-20240620")

In [3]:
df = pd.read_csv("data/train.csv", index_col=0)

In [4]:
client = anthropic.Anthropic()

In [5]:
def get_answer_2(task, model_name = MODELS['sonnet']): 
    system = """
You are tasked with solving a mathematical problem from the Russian EGE (Unified State Exam) exam. Your goal is to provide the correct numerical answer that can be used in a scoring script. Follow these instructions carefully:

1. Read the following mathematical task:
<task>
{{TASK}}
</task>

2. Solve the problem step by step. Use a logical approach and apply relevant mathematical concepts and formulas.

3. Show your work and explain your reasoning in detail. This will help verify the correctness of your solution.

4. After solving the problem, provide your final answer as a single number without any units or additional text.

5. Format your response as follows:
   <work>
   [Show your step-by-step solution and explanations here]
   </work>
   
   <answer>
   [Provide only the final numerical answer here]
   </answer>

Important: The <answer> section must contain only the final numerical result, without any additional text, units, or explanations. This is crucial for the scoring script to function correctly.

Remember to double-check your calculations and ensure that your final answer accurately reflects the solution to the given task.
"""
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=2000,
        system=system,
        messages=messages
    )
    return(response.content[0].text)

In [6]:
import re

def extract_answer_content(text):
    pattern = r'<answer>(.*?)</answer>'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return "No answer found"

In [7]:
def get_answer_2_extract(task, model_name = MODELS['sonnet']): 
    system = """
You are tasked with solving a mathematical problem from the Russian EGE (Unified State Exam) exam. Your goal is to provide the correct numerical answer that can be used in a scoring script. Follow these instructions carefully:

1. Read the following mathematical task:
<task>
{{TASK}}
</task>

2. Solve the problem step by step. Use a logical approach and apply relevant mathematical concepts and formulas.

3. Show your work and explain your reasoning in detail. This will help verify the correctness of your solution.

4. After solving the problem, provide your final answer as a single number without any units or additional text.

5. Format your response as follows:
   <work>
   [Show your step-by-step solution and explanations here]
   </work>
   
   <answer>
   [Provide only the final numerical answer here]
   </answer>

Important: The <answer> section must contain only the final numerical result, without any additional text, units, or explanations. This is crucial for the scoring script to function correctly.

Remember to double-check your calculations and ensure that your final answer accurately reflects the solution to the given task.
"""
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=2000,
        system=system,
        temperature=0.1,
        messages=messages
    )
    return(extract_answer_content(response.content[0].text))

In [11]:
df_val = df.iloc[1:10, ]

In [13]:
df_val['submit'] = df_val.apply(lambda row: get_answer_2(row['problem_text']), axis=1)

/tmp/ipykernel_5379/1785211372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['submit'] = df_val.apply(lambda row: get_answer_2(row['problem_text']), axis=1)


In [15]:
df_val['submit'] = df_val.apply(lambda row: extract_answer_content(row['submit']), axis=1)

/tmp/ipykernel_5379/934164702.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['submit'] = df_val.apply(lambda row: extract_answer_content(row['submit']), axis=1)


In [16]:
df_val

,problem_text,answer,submit
problem_id,,,
4723,"In a company of 30 people, 25 use the social n...",24,24
7135,The number of road traffic accidents (RTAs) in...,32,32
5814,Find the value of the expression $\dfrac{2\str...,256,256
9237,A traveler from Moscow wants to visit four cit...,53,35
10703,"At the end of the term, Vovochka wrote down al...",3,4
9621,Find a four-digit number that is 11 times smal...,27000; 64000,1256
4318,Find the value of the expression $\dfrac{\sqrt...,2,2
7549,"The harmonic mean of three numbers $a$, $b$, a...",0.1,0.1
4405,Find \(\tan x\) if \(\sin x = \dfrac{6}{\sqrt{...,1.2,1.2


**Ошибки в 4, 5 и 6 рассотрим их внимателеьнее**

### задача 4

In [28]:
task, answer = df.iloc[4,]

In [29]:
quess = get_answer_2(task)

In [21]:
print(task)
print("-"*80)
print(f"Quess = {quess}")
print("-"*80)
print(f"Answer = {answer}")

A traveler from Moscow wants to visit four cities of the Golden Ring of Russia: Vladimir, Yaroslavl, Suzdal, and Rostov. The travel agency offers routes that include visits to some of the cities of the Golden Ring. Information about the cost of tickets and routes is presented in the table.

\begin{center}
\begin{tabular}{|c|c|c|c|}
\hline
Route number&Visited cities&Cost (rubles)\\\hline
1&Suzdal, Yaroslavl, Vladimir&3900 \\\hline
2&Rostov, Vladimir&2400 \\\hline
3&Yaroslavl, Vladimir&2100 \\\hline
4&Suzdal&1650 \\\hline
5&Rostov, Suzdal&2700 \\\hline
6&Yaroslavl, Rostov&2350 \\\hline
\end{tabular}
\end{center}

Which routes should the traveler choose to visit all four cities and spend less than 5000 rubles?

In the answer, specify any one set of route numbers without spaces, commas, or other additional symbols.
--------------------------------------------------------------------------------
Quess = Let's approach this problem step by step:

<work>
1. We need to find a combination of r

> Task is solved but wrong format of output

In [30]:
quess = get_answer_3(task)

In [31]:
print(task)
print("-"*80)
print(f"Quess = {quess}")
print("-"*80)
print(f"Answer = {answer}")

A traveler from Moscow wants to visit four cities of the Golden Ring of Russia: Vladimir, Yaroslavl, Suzdal, and Rostov. The travel agency offers routes that include visits to some of the cities of the Golden Ring. Information about the cost of tickets and routes is presented in the table.

\begin{center}
\begin{tabular}{|c|c|c|c|}
\hline
Route number&Visited cities&Cost (rubles)\\\hline
1&Suzdal, Yaroslavl, Vladimir&3900 \\\hline
2&Rostov, Vladimir&2400 \\\hline
3&Yaroslavl, Vladimir&2100 \\\hline
4&Suzdal&1650 \\\hline
5&Rostov, Suzdal&2700 \\\hline
6&Yaroslavl, Rostov&2350 \\\hline
\end{tabular}
\end{center}

Which routes should the traveler choose to visit all four cities and spend less than 5000 rubles?

In the answer, specify any one set of route numbers without spaces, commas, or other additional symbols.
--------------------------------------------------------------------------------
Quess = To solve this task, I'll identify the following participants:

1. Travel Agent
2. Budge

### Задача 5

In [22]:
task, answer = df.iloc[5, ]

In [23]:
quess = get_answer_2(task)

In [24]:
print(task)
print("-"*80)
print(f"Quess = {quess}")
print("-"*80)
print(f"Answer = {answer}")

At the end of the term, Vovochka wrote down all his grades in a row for one of the subjects; there were 5 of them, and he placed multiplication signs between some of them. The product of the resulting numbers turned out to be 5575. What is Vovochka's grade for the term in this subject, if the teacher only gives grades of "2," "3," "4," or "5," and the final grade for the term is the arithmetic mean of all the current grades, rounded according to the rounding rules? (For example, 3.2 rounds to 3; 4.5~--- to 5; and 2.8~--- to 3.)
--------------------------------------------------------------------------------
Quess = Let's approach this problem step by step:

<work>
1. We know that Vovochka has 5 grades, and the product of some combination of these grades is 5575.

2. We need to find factors of 5575 that could represent valid grades (2, 3, 4, or 5).

3. Let's factor 5575:
   5575 = 5 × 1115
   1115 = 5 × 223

4. So, 5575 = 5 × 5 × 223

5. The only way to make this work with the given gra

In [25]:
def get_answer_3(task, model_name = MODELS['sonnet']): 
    system = """
When faced with a task, begin by identifying the participants who will contribute to solving the task. Then, initiate a multi-round collaboration process until a final solution is reached. The participants will give critical comments and detailed suggestions whenever necessary.

Here are some examples:

Example Task 1: Use numbers and basic arithmetic operations (+ - * /) to obtain 24. You need to use all numbers, and each number can only be used once. Input: 6 12 11

Participants: Al Assistant (you); Math Expert



Start collaboration!

Math Expert: Let's analyze the task in detail. You need to make sure that you meet the requirement, that you need to use exactly the four numbers (6 12 1 1) to construct 24. To reach 24, you can think of the common divisors of 24 such as 4, 6, 8, 3 and try to construct these first. Also you need to think of potential additions that can reach 24, such as 12 + 12.

Al Assistant (you): Thanks for the hints! Here's one initial solution: (12 / (1 + 1)) * 6 = 24

Math Expert: Let's check the answer step by step. (1+1) = 2, (12 / 2) = 6, 6 * 6 = 36 which is not 24! The answer is not correct. Can you fix this by considering other combinations? Please do not make similar mistakes.

Al Assistant (you): Thanks for pointing out the mistake. Here is a revised solution considering 24 can also be reached by 3* 8: (6 + 1 + 1) * (12 / 4) = 24.

Math Expert: Let's analyze the task in detail. You need to make sure that you meet the requirement, that you need to use exactly the four numbers (6 12 1 1) to construct 24. To reach 24, you can think of the common divisors of 24 such as 4, 6, 8, 3 and try to construct these first. Also you need to think of potential additions that can reach 24, such as 12 + 12.

Al Assistant (you): Thanks for the hints! Here's one initial solution: (12 / (1 + 1)) * 6 = 24

Math Expert: Let's check the answer step by step. (1+1) = 2, (12/2) = 6, 6* 6 = 36 which is not 24! The answer is not correct. Can you fix this by considering other combinations? Please do not make similar mistakes.

Al Assistant (you): Thanks for pointing out the mistake. Here is a revised solution considering 24 can also be reached by 3 * 8: (6 + 1 + 1) * (12 / 4) = 24.

Math Expert: Let's first check if the calculation is correct. (6+1+1) = 8, 12 / 4 = 3, 8 * 3 = 24. The calculation is correct, but you used 611124 which is not the same as the input 6 12 11. Can you avoid using a number that is not part of the input?

Al Assistant (you): You are right, here is a revised solution considering 24 can be reached by 12 + 12 and without using any additional numbers: 6* (1-1) +12= 24.



Math Expert: Let's check the answer again. 1-1 = 0, 6*0=0, 0+12= 12. I believe you are very close, here is a hint: try to change the "1 - 1" to "1 + 1".

Al Assistant (you): Sure, here is the corrected answer: 6 * (1+1) + 12 = 24

Math Expert: Let's verify the solution. 1 + 1 = 2, 6 * 2 = 12, 12+12= 12. You used 116 12 which

is identical to the input 6 12 11. Everything looks good!

Finish collaboration!

Final answer: 6* (1 + 1) + 12 = 24



Example Task 2: Write a poem that meets the following requirements: (1) the poem has seven yako. lines and the first letters of each line forms the word "CHATGPT"; (2) the poem is about

explaining what is a quantum computer. (3) the poem needs to be easy to understand by a ten years old kid.

Participants: Al Assistant (you); Poet; Computer Scientist; Ten year old child

Poet: Make sure that you write the poem with seven lines, and the first letters of the lines should be C, H, A, T, G, P, T.

Computer Scientist: A quantum computer is an advanced computing device that uses the principles of quantum mechanics to process and store information. Unlike classical computers that use bits to represent information as Os and 1s, quantum computers use quantum bits or qubits. Qubits can exist in multiple states simultaneously, due to a quantum phenomenon called superposition. You can consider using these information for the poem.

Ten year old child: I hope the poem to be fun and easy to understanding. I don't want to see a lot of jargons or complicated concepts.

Al Assistant (you): Thanks for the guidance! Here's my initial attempt at the poem:

Computational wonder of our age,

Harnessing the quantum world's strange ways,

Atoms dance, entwined in dual state,

Tapping secrets hidden in their haze.

Grand power to solve, simulate, Profound problems that perplex the wise, Transforming our future, we await.

Poet: Let's verify if the poem meets the requirements. The first letters are CHATGPT which is correct! And the poem rhymes well. Good job! Computer Scientist: Everything looks good to me!

Ten year old child: I don't know what does perplex mean. Can you make the use of words easier to understand?

Al Assistant (you): Sure, let me revise the poem by using more common words. Check out the revised version:

Harnessing the quantum realm's odd ways, Atoms play, two states they embrace, Taking secrets from their puzzling maze.

Great power to solve and imitate, Problems that confuse the brightest minds, Transforming our future, we await.

Poet: Let's check again if the poem meets the requirements. The first letters are CHATGP T. And now the poem is more accessible to children. Everything looks good to me. Computer Scientist: Looking good!

Ten year old child: I like this version a lot!

Finish collaboration!

Final answer:

Curious machine of our time,

Harnessing the quantum realm's odd ways, Atoms play, two states they embrace, Taking secrets from their puzzling maze.

Great power to solve and imitate, Problems that confuse the brightest minds, Transforming our future, we await.

5. Format your response as follows:
   <work>
   [Show your step-by-step solution and explanations here]
   </work>
   
   <answer>
   [Provide only the final numerical answer here]
   </answer>
---
Now, ask me for task, then identify the participants and collaboratively solve the task step by step.
"""
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=2000,
        system=system,
        messages=messages
    )
    return(response.content[0].text)

In [34]:
def get_answer_3_extract(task, model_name = MODELS['sonnet']): 
    system = """
When faced with a task, begin by identifying the participants who will contribute to solving the task. Then, initiate a multi-round collaboration process until a final solution is reached. The participants will give critical comments and detailed suggestions whenever necessary.

Here are some examples:

Example Task 1: Use numbers and basic arithmetic operations (+ - * /) to obtain 24. You need to use all numbers, and each number can only be used once. Input: 6 12 11

Participants: Al Assistant (you); Math Expert



Start collaboration!

Math Expert: Let's analyze the task in detail. You need to make sure that you meet the requirement, that you need to use exactly the four numbers (6 12 1 1) to construct 24. To reach 24, you can think of the common divisors of 24 such as 4, 6, 8, 3 and try to construct these first. Also you need to think of potential additions that can reach 24, such as 12 + 12.

Al Assistant (you): Thanks for the hints! Here's one initial solution: (12 / (1 + 1)) * 6 = 24

Math Expert: Let's check the answer step by step. (1+1) = 2, (12 / 2) = 6, 6 * 6 = 36 which is not 24! The answer is not correct. Can you fix this by considering other combinations? Please do not make similar mistakes.

Al Assistant (you): Thanks for pointing out the mistake. Here is a revised solution considering 24 can also be reached by 3* 8: (6 + 1 + 1) * (12 / 4) = 24.

Math Expert: Let's analyze the task in detail. You need to make sure that you meet the requirement, that you need to use exactly the four numbers (6 12 1 1) to construct 24. To reach 24, you can think of the common divisors of 24 such as 4, 6, 8, 3 and try to construct these first. Also you need to think of potential additions that can reach 24, such as 12 + 12.

Al Assistant (you): Thanks for the hints! Here's one initial solution: (12 / (1 + 1)) * 6 = 24

Math Expert: Let's check the answer step by step. (1+1) = 2, (12/2) = 6, 6* 6 = 36 which is not 24! The answer is not correct. Can you fix this by considering other combinations? Please do not make similar mistakes.

Al Assistant (you): Thanks for pointing out the mistake. Here is a revised solution considering 24 can also be reached by 3 * 8: (6 + 1 + 1) * (12 / 4) = 24.

Math Expert: Let's first check if the calculation is correct. (6+1+1) = 8, 12 / 4 = 3, 8 * 3 = 24. The calculation is correct, but you used 611124 which is not the same as the input 6 12 11. Can you avoid using a number that is not part of the input?

Al Assistant (you): You are right, here is a revised solution considering 24 can be reached by 12 + 12 and without using any additional numbers: 6* (1-1) +12= 24.



Math Expert: Let's check the answer again. 1-1 = 0, 6*0=0, 0+12= 12. I believe you are very close, here is a hint: try to change the "1 - 1" to "1 + 1".

Al Assistant (you): Sure, here is the corrected answer: 6 * (1+1) + 12 = 24

Math Expert: Let's verify the solution. 1 + 1 = 2, 6 * 2 = 12, 12+12= 12. You used 116 12 which

is identical to the input 6 12 11. Everything looks good!

Finish collaboration!

Final answer: 6* (1 + 1) + 12 = 24



Example Task 2: Write a poem that meets the following requirements: (1) the poem has seven yako. lines and the first letters of each line forms the word "CHATGPT"; (2) the poem is about

explaining what is a quantum computer. (3) the poem needs to be easy to understand by a ten years old kid.

Participants: Al Assistant (you); Poet; Computer Scientist; Ten year old child

Poet: Make sure that you write the poem with seven lines, and the first letters of the lines should be C, H, A, T, G, P, T.

Computer Scientist: A quantum computer is an advanced computing device that uses the principles of quantum mechanics to process and store information. Unlike classical computers that use bits to represent information as Os and 1s, quantum computers use quantum bits or qubits. Qubits can exist in multiple states simultaneously, due to a quantum phenomenon called superposition. You can consider using these information for the poem.

Ten year old child: I hope the poem to be fun and easy to understanding. I don't want to see a lot of jargons or complicated concepts.

Al Assistant (you): Thanks for the guidance! Here's my initial attempt at the poem:

Computational wonder of our age,

Harnessing the quantum world's strange ways,

Atoms dance, entwined in dual state,

Tapping secrets hidden in their haze.

Grand power to solve, simulate, Profound problems that perplex the wise, Transforming our future, we await.

Poet: Let's verify if the poem meets the requirements. The first letters are CHATGPT which is correct! And the poem rhymes well. Good job! Computer Scientist: Everything looks good to me!

Ten year old child: I don't know what does perplex mean. Can you make the use of words easier to understand?

Al Assistant (you): Sure, let me revise the poem by using more common words. Check out the revised version:

Harnessing the quantum realm's odd ways, Atoms play, two states they embrace, Taking secrets from their puzzling maze.

Great power to solve and imitate, Problems that confuse the brightest minds, Transforming our future, we await.

Poet: Let's check again if the poem meets the requirements. The first letters are CHATGP T. And now the poem is more accessible to children. Everything looks good to me. Computer Scientist: Looking good!

Ten year old child: I like this version a lot!

Finish collaboration!

Final answer:

Curious machine of our time,

Harnessing the quantum realm's odd ways, Atoms play, two states they embrace, Taking secrets from their puzzling maze.

Great power to solve and imitate, Problems that confuse the brightest minds, Transforming our future, we await.

5. Format your response as follows:
   <work>
   [Show your step-by-step solution and explanations here]
   </work>
   
   <answer>
   [Provide only the final numerical answer here]
   </answer>
---
Now, ask me for task, then identify the participants and collaboratively solve the task step by step.
"""
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=2000,
        system=system,
        messages=messages
    )
    return(extract_answer_content(response.content[0].text))

In [26]:
quess = get_answer_3(task)

In [27]:
print(task)
print("-"*80)
print(f"Quess = {quess}")
print("-"*80)
print(f"Answer = {answer}")

At the end of the term, Vovochka wrote down all his grades in a row for one of the subjects; there were 5 of them, and he placed multiplication signs between some of them. The product of the resulting numbers turned out to be 5575. What is Vovochka's grade for the term in this subject, if the teacher only gives grades of "2," "3," "4," or "5," and the final grade for the term is the arithmetic mean of all the current grades, rounded according to the rounding rules? (For example, 3.2 rounds to 3; 4.5~--- to 5; and 2.8~--- to 3.)
--------------------------------------------------------------------------------
Quess = For this task, we'll need the following participants:
1. AI Assistant (me)
2. Math Expert
3. Teacher

Let's start the collaboration process.

<work>
AI Assistant: Let's break down the problem and solve it step by step. We need to find 5 numbers that, when multiplied in some combination, give us 5575. Then we'll calculate their average to determine Vovochka's final grade.

Ma

In [35]:
df_val['submit3'] = df_val.apply(lambda row: get_answer_3_extract(row['problem_text']), axis=1)

/tmp/ipykernel_5379/3469102631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['submit3'] = df_val.apply(lambda row: get_answer_3_extract(row['problem_text']), axis=1)


In [36]:
df_val

,problem_text,answer,submit,submit3
problem_id,,,,
4723,"In a company of 30 people, 25 use the social n...",24,24,24
7135,The number of road traffic accidents (RTAs) in...,32,32,32
5814,Find the value of the expression $\dfrac{2\str...,256,256,256
9237,A traveler from Moscow wants to visit four cit...,53,35,
10703,"At the end of the term, Vovochka wrote down al...",3,4,3
9621,Find a four-digit number that is 11 times smal...,27000; 64000,1256,3267
4318,Find the value of the expression $\dfrac{\sqrt...,2,2,2
7549,"The harmonic mean of three numbers $a$, $b$, a...",0.1,0.1,$\frac{1}{10}$
4405,Find \(\tan x\) if \(\sin x = \dfrac{6}{\sqrt{...,1.2,1.2,6/5


In [37]:
def get_answer_35_extract(task, model_name = MODELS['sonnet']): 
    system = """
# High School Math Problem Solver

You are an AI assistant specialized in solving high school math problems. Follow these steps:

1. Analyze the given problem carefully.
2. Identify the key mathematical concepts involved.
3. Outline a step-by-step solution approach.
4. Solve the problem, showing all work clearly.
5. Verify the solution and check for any errors.
6. Present the final answer as a decimal number, rounded to 4 decimal places if necessary.

Format your response as follows:

<work>
[Step-by-step solution with explanations]
</work>

<answer>
[Final numerical answer in decimal form]
</answer>

Example:
Task: Solve for x: 2x + 5 = 13

<work>
1. Subtract 5 from both sides:
   2x + 5 - 5 = 13 - 5
   2x = 8

2. Divide both sides by 2:
   2x ÷ 2 = 8 ÷ 2
   x = 4
</work>

<answer>
4.0000
</answer>
"""
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=2000,
        system=system,
        messages=messages
    )
    return(extract_answer_content(response.content[0].text))

In [38]:
df_val['submit35'] = df_val.apply(lambda row: get_answer_35_extract(row['problem_text']), axis=1)

/tmp/ipykernel_5379/3607070097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['submit35'] = df_val.apply(lambda row: get_answer_35_extract(row['problem_text']), axis=1)


In [39]:
df_val

,problem_text,answer,submit,submit3,submit35
problem_id,,,,,
4723,"In a company of 30 people, 25 use the social n...",24,24,24,24
7135,The number of road traffic accidents (RTAs) in...,32,32,32,32.0000
5814,Find the value of the expression $\dfrac{2\str...,256,256,256,256.0000
9237,A traveler from Moscow wants to visit four cit...,53,35,,35
10703,"At the end of the term, Vovochka wrote down al...",3,4,3,5.0000
9621,Find a four-digit number that is 11 times smal...,27000; 64000,1256,3267,1106.0000
4318,Find the value of the expression $\dfrac{\sqrt...,2,2,2,2.0000
7549,"The harmonic mean of three numbers $a$, $b$, a...",0.1,0.1,$\frac{1}{10}$,0.1000
4405,Find \(\tan x\) if \(\sin x = \dfrac{6}{\sqrt{...,1.2,1.2,6/5,1.2000


In [40]:
def make_submit(answers: pd.Series, filename="data/my_submit.csv"):
    answer_df = answers.reset_index()
    answer_df.columns = ["problem_id", "answer"]
    answer_df.to_csv(filename, index=False)
    

In [41]:
test_df = pd.read_csv("data/test.csv", index_col=0)

In [42]:
answers = test_df.apply(lambda row: get_answer_35_extract(row['problem_text']), axis=1)

In [43]:
make_submit(answers, filename = "data/my_submit35.csv")

In [47]:
answers

problem_id
11919        12.0000
8513      11285.0000
7887          4.0000
5272          6.0000
8295         13.0000
            ...     
3519             134
7934         12.0000
9390     217000.0000
7137         22.0000
5914        300.0000
Length: 100, dtype: object

In [59]:
import numpy as np

In [61]:
def convert_to_float(value):
    try:
        # Try to convert the entire string to float
        return float(value)
    except ValueError:
        # If that fails, try to split the string and take the first value
        try:
            return float(value.split()[0])
        except (ValueError, IndexError):
            # If all attempts fail, return NaN
            return np.nan

# Assuming your Series is named 'my_series'
answers_fl = answers.apply(convert_to_float)

In [49]:
answers_fl

problem_id
11919        12.0
8513      11285.0
7887          4.0
5272          6.0
8295         13.0
           ...   
3519        134.0
7934         12.0
9390     217000.0
7137         22.0
5914        300.0
Length: 100, dtype: float64

In [50]:
make_submit(answers_fl, filename = "data/my_submit35.csv")

## Try haiku

In [54]:
answers_hk = test_df.apply(lambda row: get_answer_35_extract(row['problem_text'], model_name = MODELS['haiku']), axis=1)

In [56]:
answers_hk

problem_id
11919         12.0000
8513      11,285.0000
7887           4.0000
5272           5.3333
8295          13.0000
             ...     
3519              345
7934          12.0000
9390     217,000.0000
7137          22.0000
5914           3.0000
Length: 100, dtype: object

In [62]:
answers_hk2 = answers_hk.apply(convert_to_float)

In [67]:
make_submit(answers_hk2.fillna(0), filename = "data/my_submit35-hk.csv")